In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/math-pdf/math_pdf.pdf
/kaggle/input/my-pdf-digital/Digital_Fluency_Module3_asd.pdf


In [12]:
!apt-get update
!apt-get install -y postgresql postgresql-contrib postgresql-server-dev-all
!service postgresql start
!pip install phidata mistral mistralai openai pypdf "psycopg[binary]" sqlalchemy pgvector groq bertopic psycopg


0% [Working]

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease         
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                                              
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease                                    
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease                                      
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease                                    
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease                                          
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease     
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/so

In [13]:
!git clone https://github.com/pgvector/pgvector.git
!cd pgvector && make && make install

fatal: destination path 'pgvector' already exists and is not an empty directory.
make: Nothing to be done for 'all'.
/bin/mkdir -p '/usr/lib/postgresql/14/lib'
/bin/mkdir -p '/usr/share/postgresql/14/extension'
/bin/mkdir -p '/usr/share/postgresql/14/extension'
/usr/bin/install -c -m 755  vector.so '/usr/lib/postgresql/14/lib/vector.so'
/usr/bin/install -c -m 644 .//vector.control '/usr/share/postgresql/14/extension/'
/usr/bin/install -c -m 644 .//sql/vector--0.1.0--0.1.1.sql .//sql/vector--0.1.1--0.1.3.sql .//sql/vector--0.1.3--0.1.4.sql .//sql/vector--0.1.4--0.1.5.sql .//sql/vector--0.1.5--0.1.6.sql .//sql/vector--0.1.6--0.1.7.sql .//sql/vector--0.1.7--0.1.8.sql .//sql/vector--0.1.8--0.2.0.sql .//sql/vector--0.2.0--0.2.1.sql .//sql/vector--0.2.1--0.2.2.sql .//sql/vector--0.2.2--0.2.3.sql .//sql/vector--0.2.3--0.2.4.sql .//sql/vector--0.2.4--0.2.5.sql .//sql/vector--0.2.5--0.2.6.sql .//sql/vector--0.2.6--0.2.7.sql .//sql/vector--0.2.7--0.3.0.sql .//sql/vector--0.3.0--0.3.1.sql .//sql/

In [14]:
# Switch to the postgres user and execute PostgreSQL commands
# Create a new database named 'kaggle_db'
!sudo -u postgres psql -c "CREATE DATABASE kaggle_db;"
# Create a new user 'kaggle_user' with a password
!sudo -u postgres psql -c "CREATE USER kaggle_user WITH PASSWORD 'kaggle_password';"
# Configure the 'kaggle_user' role with desired settings
!sudo -u postgres psql -c "ALTER ROLE kaggle_user SET client_encoding TO 'utf8';"
!sudo -u postgres psql -c "ALTER ROLE kaggle_user SET default_transaction_isolation TO 'read committed';"
!sudo -u postgres psql -c "ALTER ROLE kaggle_user SET timezone TO 'UTC';"
# Grant privileges to 'kaggle_user' for the 'kaggle_db' database
!sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE kaggle_db TO kaggle_user;"
# Create Extension if not available 
!sudo -u postgres psql -d kaggle_db -c "CREATE EXTENSION IF NOT EXISTS vector;"


ERROR:  database "kaggle_db" already exists
ERROR:  role "kaggle_user" already exists
ALTER ROLE
ALTER ROLE
ALTER ROLE
GRANT
NOTICE:  extension "vector" already exists, skipping
CREATE EXTENSION


In [15]:
import os
import nltk
import torch
import psycopg
import psycopg2

from time import sleep
from hashlib import md5
from phi.agent import Agent
from bertopic import BERTopic
from nltk.corpus import stopwords
from phi.model.mistral import MistralChat
from phi.vectordb.pgvector import PgVector
from pgvector.psycopg import register_vector
from transformers import AutoTokenizer, AutoModel
from phi.knowledge.pdf import PDFKnowledgeBase, PDFReader
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
# PDF processer Q&A Generator
class PDFProcessor:
    def __init__(self, db_config, mistral_api_key):
        self.conn = psycopg.connect(**db_config)
        register_vector(self.conn)

        # Initialize embedding model
        self.tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
        self.model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

        # Initialize the Agent with MistralChat model
        self.agent = Agent(
            model=MistralChat(
                id="mistral-large-latest",
                api_key=mistral_api_key,
            ),
            markdown=True
        )

    def setup_database(self):
        query = f"""
            CREATE TABLE IF NOT EXISTS {self.doc_db_name} (
                id SERIAL PRIMARY KEY,
                content TEXT NOT NULL,
                embedding VECTOR(384),
                content_hash VARCHAR(64) NOT NULL UNIQUE,
                metadata JSONB,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
            """
        with self.conn.cursor() as cur:
            cur.execute(query)
            self.conn.commit()

    def get_embeddings(self, text: str) -> list:
        tokens = self.tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            embeddings = self.model(**tokens).last_hidden_state.mean(dim=1)
        return embeddings.squeeze(0).tolist()

    def load_pdf(self, pdf_path: str, max_pages: int = None):
        from phi.knowledge.pdf import PDFReader

        reader = PDFReader(chunk=False)  # Changed to False to get full pages
        documents = reader.read(pdf_path)
        self.doc_db_name = pdf_path.split("/")[-1][:-4]
        print(self.doc_db_name)
        self.setup_database()
        sleep(1)
        # Limit the number of pages if specified
        if max_pages is not None:
            documents = documents[:max_pages]

        for i, doc in enumerate(documents, start=1):
            content_hash = md5(doc.content.encode("utf-8")).hexdigest()
            
            # Check if page already exists
            with self.conn.cursor() as cur:
                query = f"SELECT 1 FROM {self.doc_db_name} WHERE content_hash = %s"
                cur.execute(query, (content_hash,))
                if cur.fetchone():
                    print(f"Page {i} already exists in the database.")
                    continue

            # Generate and store embedding
            try:
                embedding = self.get_embeddings(doc.content)
                # table_name = self.doc_db_name  # Ensure this is validated or sanitized!
                query = f"""
                INSERT INTO {self.doc_db_name} (content, embedding, content_hash, metadata)
                VALUES (%s, %s, %s, %s)
                """
                metadata = '{"source": "pdf", "page": ' + str(i) + '}'
                with self.conn.cursor() as cur:
                    cur.execute(query, (doc.content, embedding, content_hash, metadata))
                    self.conn.commit()
                print(f"Inserted page {i}: Successfully processed.")
            except Exception as e:
                print(f"Error processing page {i}: {str(e)}")
                continue

    def generate_mcqs(self,num_questions: int = 5,topics: list=[], level: str = "Medium") -> str:
        # self.doc_db_name = pdf_path.split("/")[-1].strip(".pdf")
        # print(self.doc_db_name)
        query = f"SELECT content FROM {self.doc_db_name}"
        # Get some content from the database
        with self.conn.cursor() as cur:
            cur.execute(query)
            contents = [row[0] for row in cur.fetchall()]

        content_text = " ".join(contents)
        # print(content_text)

        prompt = f"""
        You are an AI assistant helping the user generate multiple-choice questions (MCQs) based on the following text:
        'from the PDF: {content_text}' and 
        'Use the Following Topics {topics} to Get the Questions if topics is empty only then random Question from the PDF content'
        Please generate {num_questions} questions in the {level} category. Each question should have:
        - A clear question
        - Four answer options (labeled A, B, C, D)
        - The correct answer clearly indicated
        Expected output JSON format:
        [[{{"Question":"Question; A) Option 1; B) Option 2; C) Option 3; D) Option 4"}},{{"Answer":"*) Correct Option"}}]]
        Strict Note: Don't go out of the Pdf Content.
        """

        try:
            # Use the agent to generate a response
            response = self.agent.run(prompt)
            return response.content
        except Exception as e:
            print(f"Error generating MCQs: {str(e)}")
            return "Failed to generate MCQs."

    def generate_QA1(self, num_questions: int = 5,topics: list=[], level: str = "Medium") -> str:
        # Get some content from the database
        query = f"SELECT content FROM {self.doc_db_name}"
        with self.conn.cursor() as cur:
            cur.execute(query)
            contents = [row[0] for row in cur.fetchall()]

        content_text = " ".join(contents)
        # print(content_text)

        prompt = f"""
        You are an AI assistant helping the user generate questions & answer with one or two sentences each based on the following text:
        'from the PDF: {content_text}' and 
        'Use the Following Topics {topics} to Get the Questions if topics is empty only then random Question from the PDF content'
        Please generate {num_questions} with Category like Easy,Medium and Hard kindly use {level} category to generate Question and answer with 1-2 Sentences from the text. Each question should have:
        - A clear question
        - A clear answer with 1-2 sentences
        Expexted output json format.
        [{{"category":level,"QA_*":{{"Question":*question,"Answer":*answer}}}}]
        Strict Note: Don't go out of the Pdf Content.
        """

        try:
            # Use the agent to generate a response
            response = self.agent.run(prompt)
            return response.content
        except Exception as e:
            print(f"Error generating MCQs: {str(e)}")
            return "Failed to generate MCQs."


In [17]:
# PDF processer for Topic extraction.
class PDFProcessorWithTopicModeling(PDFProcessor):
    def __init__(self, db_config, mistral_api_key):
        super().__init__(db_config, mistral_api_key)
        self.stop_words = set(stopwords.words("english"))
        self.top_10_topics = {"TF-IDF": [], "BERTopic": []}  # Initialize top 10 topics

    def preprocess_text(self, documents):
        """Preprocess text by removing stop words and applying basic cleaning."""
        return [
            " ".join(
                word for word in doc.split() if word.lower() not in self.stop_words
            )
            for doc in documents
        ]

    def extract_topics_tfidf(self, top_n=10):
        """Extract topics using TF-IDF."""
        query = f"SELECT content FROM {self.doc_db_name}"
        with self.conn.cursor() as cur:
            cur.execute(query)
            documents = [row[0] for row in cur.fetchall()]

        if not documents:
            print("No documents found for topic modeling.")
            return []

        # Preprocess text
        preprocessed_docs = self.preprocess_text(documents)

        # TF-IDF vectorization
        vectorizer = TfidfVectorizer(max_features=5000)
        tfidf_matrix = vectorizer.fit_transform(preprocessed_docs)
        feature_names = vectorizer.get_feature_names_out()

        # Get top terms for each document
        top_terms = []
        for i in range(tfidf_matrix.shape[0]):
            row = tfidf_matrix[i].toarray()[0]
            top_indices = row.argsort()[-top_n:][::-1]
            terms = [(feature_names[idx], row[idx]) for idx in top_indices]
            top_terms.append(terms)
            
        # Store the top topics in self.top_10_topics
        self.top_10_topics["TF-IDF"] = top_terms

        return top_terms

    def extract_topics_bertopic(self):
        """Extract topics using BERTopic."""
        query = f"SELECT content FROM {self.doc_db_name}"
        with self.conn.cursor() as cur:
            cur.execute(query)
            documents = [row[0] for row in cur.fetchall()]

        if not documents:
            print("No documents found for topic modeling.")
            return []

        # Preprocess text
        preprocessed_docs = self.preprocess_text(documents)

        # Initialize BERTopic
        topic_model = BERTopic()
        topics, probs = topic_model.fit_transform(preprocessed_docs)

        # Get top topics
        topics_summary = topic_model.get_topic_info()

        # Store the top topics in self.top_10_topics
        self.top_10_topics["BERTopic"] = topics_summary

        return topics_summary
        
    def display_topics(self):
        """Display topics from both TF-IDF and BERTopic."""
        print("\nExtracting Topics with TF-IDF:")
        tfidf_topics = self.extract_topics_tfidf(top_n=10)
        for i, terms in enumerate(tfidf_topics, start=1):
            print(f"Document {i}:")
            for term, score in terms:
                print(f"- {term} (Score: {score:.4f})")
        
        print("\nExtracting Topics with BERTopic:")
        bertopic_summary = self.extract_topics_bertopic()
        print(bertopic_summary)


In [18]:
# Pre-requisite 
mistral_api_key = "" # Add your API Key here 
# Database configuration
db_config = {
    "dbname": "kaggle_db",
    "user": "kaggle_user",
    "password": "kaggle_password",
    "host": "127.0.0.1",
    "port": "5432"
}

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Initial PDF

In [19]:
topic_processor = PDFProcessorWithTopicModeling(db_config, mistral_api_key)

pdf_path = "/kaggle/input/math-pdf/math_pdf.pdf"
topic_processor.load_pdf(pdf_path, max_pages=10)

# Display topics using TF-IDF and BERTopic
topic_processor.display_topics()

INFO     Reading: math_pdf

math_pdf
Page 1 already exists in the database.
Page 2 already exists in the database.
Page 3 already exists in the database.
Page 4 already exists in the database.
Page 5 already exists in the database.
Page 6 already exists in the database.
Page 7 already exists in the database.
Page 8 already exists in the database.
Page 9 already exists in the database.
Page 10 already exists in the database.

Extracting Topics with TF-IDF:
Document 1:
- functions (Score: 0.2846)
- relations (Score: 0.2033)
- class (Score: 0.1799)
- xi (Score: 0.1690)
- kind (Score: 0.1515)
- quantities (Score: 0.1515)
- final (Score: 0.1515)
- beauty (Score: 0.1515)
- connection (Score: 0.1515)
- link (Score: 0.1515)
Document 2:
- relation (Score: 0.4604)
- empty (Score: 0.3148)
- a2 (Score: 0.2676)
- a1 (Score: 0.2676)
- school (Score: 0.2341)
- set (Score: 0.2337)
- universal (Score: 0.2099)
- called (Score: 0.1561)
- relations (Score: 0.1408)
- a3 (Score: 0.1338)
Document 3:
- l1 (Score: 0.4013)
- t1 (Score: 0.

In [20]:
# Flatten the topics across all documents
all_terms = [(term, score) for doc in topic_processor.top_10_topics["TF-IDF"] for term, score in doc]

# Sort terms by score in descending order
sorted_terms = sorted(all_terms, key=lambda x: x[1], reverse=True)

# Get the top 10 topics
top_10_topics = sorted_terms[:10]
top_topics = []
print("Top 10 Topics by Score:")
for term, score in top_10_topics:
    if term not in top_topics:
        print(f"{term}: {score:.4f}")
        top_topics.append(term)
    if len(top_topics)==3:# Here it the Top 3 toopic selected 
        break

Top 10 Topics by Score:
one: 0.6391
function: 0.5169
relation: 0.4604


In [21]:
processor = PDFProcessor(db_config, mistral_api_key)

# Process PDF and limit to thefirst 10 pages
# pdf_path = "/kaggle/input/my-pdf-digital/Digital_Fluency_Module3_asd.pdf"
processor.load_pdf(pdf_path, max_pages=10)

INFO     Reading: math_pdf

math_pdf
Page 1 already exists in the database.
Page 2 already exists in the database.
Page 3 already exists in the database.
Page 4 already exists in the database.
Page 5 already exists in the database.
Page 6 already exists in the database.
Page 7 already exists in the database.
Page 8 already exists in the database.
Page 9 already exists in the database.
Page 10 already exists in the database.


# MCQ : No of MCQS 5, Level: Medium

In [22]:
# Generate MCQs
#top_topics = [] # you can ADD custom Topics Here as a list Ex: ["Romeo","Juliet","Willim Shakespeare"]
mcqs = processor.generate_mcqs(num_questions=5,topics=top_topics, level="Medium")
print("\nGenerated MCQs:\n", mcqs)



Generated MCQs:
 ```json
[
  [
    {
      "Question": "Which of the following relations is an equivalence relation?\nA) R = {(a, b) : a is sister of b}\nB) R = {(a, b) : a is brother of b}\nC) R = {(a, b) : a is father of b}\nD) R = {(a, b) : a and b live in the same locality}"
    },
    {
      "Answer": "D) R = {(a, b) : a and b live in the same locality}"
    }
  ],
  [
    {
      "Question": "Which of the following functions is both one-one and onto?\nA) f : N → N, given by f(x) = x^2\nB) f : R → R, given by f(x) = 2x\nC) f : N → N, given by f(1) = f(2) = 1 and f(x) = x - 1 for x > 2\nD) f : R → R, given by f(x) = x^2"
    },
    {
      "Answer": "B) f : R → R, given by f(x) = 2x"
    }
  ],
  [
    {
      "Question": "Which of the following relations is reflexive but neither symmetric nor transitive?\nA) R = {(1, 1), (2, 2), (3, 3), (1, 2), (2, 3)}\nB) R = {(a, b) : a is wife of b}\nC) R = {(a, b) : a is exactly 7 cm taller than b}\nD) R = {(a, b) : a – b is an integer}"
   

# MCQ : No of MCQS 5, Level: Hard

In [23]:
# Generate MCQs
mcqs = processor.generate_mcqs(num_questions=5,topics=top_topics, level="Hard")
print("\nGenerated MCQs:\n", mcqs)


Generated MCQs:
 ```json
[
  [
    {
      "Question": "Which of the following relations is an equivalence relation?\nA) R = {(a, b) : a is sister of b}\nB) R = {(a, b) : a is brother of b}\nC) R = {(a, b) : a is father of b}\nD) R = {(a, b) : a and b live in the same locality}"
    },
    {
      "Answer": "D) R = {(a, b) : a and b live in the same locality}"
    }
  ],
  [
    {
      "Question": "Consider the relation R in the set Z of integers given by R = {(a, b) : 2 divides a – b}. This relation is:\nA) Reflexive and symmetric but not transitive\nB) Symmetric and transitive but not reflexive\nC) Reflexive, symmetric, and transitive\nD) Neither reflexive, symmetric, nor transitive"
    },
    {
      "Answer": "C) Reflexive, symmetric, and transitive"
    }
  ],
  [
    {
      "Question": "Which of the following functions is both one-one and onto?\nA) f : N → N, given by f(x) = x²\nB) f : R → R, given by f(x) = 2x\nC) f : N → N, given by f(1) = f(2) = 1 and f(x) = x – 1 for x > 

# MCQ : No of Question & Answer(1-2 Sentence) 5, Level: Medium

In [24]:
# Generate MCQs
mcqs = processor.generate_QA1(num_questions=5,topics=top_topics, level="Medium")
print("\nGenerated Question & Answer(1-2 Sentence):\n", mcqs)


Generated Question & Answer(1-2 Sentence):
 ```json
[
  {
    "category": "Medium",
    "QA_1": {
      "Question": "What is an empty relation in a set A?",
      "Answer": "An empty relation in a set A is a relation where no element of A is related to any other element of A, i.e., R = φ ⊂ A × A."
    }
  },
  {
    "category": "Medium",
    "QA_2": {
      "Question": "What is a universal relation in a set A?",
      "Answer": "A universal relation in a set A is a relation where each element of A is related to every other element of A, i.e., R = A × A."
    }
  },
  {
    "category": "Medium",
    "QA_3": {
      "Question": "What does it mean for a relation to be symmetric?",
      "Answer": "A relation R in a set A is symmetric if (a1, a2) ∈ R implies (a2, a1) ∈ R, for all a1, a2 ∈ A."
    }
  },
  {
    "category": "Medium",
    "QA_4": {
      "Question": "What is an equivalence relation?",
      "Answer": "A relation R in a set A is an equivalence relation if it is reflexive, sy

# MCQ : No of Question & Answer(1-2 Sentence) 5, Level: Hard

In [25]:
# Generate MCQs
mcqs = processor.generate_QA1(num_questions=5,topics=top_topics, level="Hard")
print("\nGenerated Question & Answer(1-2 Sentence):\n", mcqs)


Generated Question & Answer(1-2 Sentence):
 ```json
[
  {
    "category": "Hard",
    "QA_1": {
      "Question": "How is the concept of a relation in mathematics derived from its English language meaning?",
      "Answer": "The concept of a relation in mathematics is derived from the English language meaning, where two objects or quantities are related if there is a recognizable connection or link between them."
    }
  },
  {
    "category": "Hard",
    "QA_2": {
      "Question": "What does it mean for a relation to be reflexive, symmetric, and transitive?",
      "Answer": "A relation R in a set A is reflexive if (a, a) ∈ R for every a ∈ A, symmetric if (a1, a2) ∈ R implies (a2, a1) ∈ R, and transitive if (a1, a2) ∈ R and (a2, a3) ∈ R implies (a1, a3) ∈ R."
    }
  },
  {
    "category": "Hard",
    "QA_3": {
      "Question": "What is an equivalence relation and what properties must it satisfy?",
      "Answer": "An equivalence relation is a relation that is reflexive, symmetric,

In [ ]:


def check_docs():
    # Database connection settings
    db_config = {
        "dbname": "kaggle_db",
        "user": "kaggle_user", 
        "password": "kaggle_password",
        "host": "127.0.0.1",
        "port": "5432"
    }
    
    conn = psycopg2.connect(**db_config)
    try:
        with conn.cursor() as cur:
            # Get total count
            cur.execute("SELECT COUNT(*) FROM {doc_db_name}")  # Changed table name to {doc_db_name}
            total = cur.fetchone()[0]
            print(f"\nTotal documents: {total}")

            # Get first document
            cur.execute("SELECT content FROM {doc_db_name} LIMIT 1")  # Changed table name to {doc_db_name}
            doc = cur.fetchone()
            if doc:
                print("\nFirst document content:")
                print(doc[0])
            else:
                print("\nNo documents found")
                
    finally:
        conn.close()

if __name__ == "__main__":
    check_docs()